In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
import glob
import numpy as np
from PIL import Image
from sklearn.metrics import pairwise_distances, roc_auc_score
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm
import sys
sys.path.append("../")
from inception_resnet_v1 import inference

In [2]:
def prewhiten(x):
    """
    A helper function to whiten an image, or a batch of images.
    Args:
        x: An image or batch of images.
    """
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        print(x.ndim)
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
#     std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    """
    Normalizes an embedding to have unit length in the l2 metric.
    Args:
        x: A batch of numpy embeddings
    """
    output = x / np.sqrt(np.maximum(np.sum(np.square(x),
                                           axis=axis,
                                           keepdims=True),
                                    epsilon))
    return output

In [3]:
class PersonImages:
    def __init__(self, person_name):
        self.clean_folder = ""
        self.clean_images = []
        self.adversarial_images = []
        self.orig_mean = None
        self.orig_std = None
        self.orig_paths = []
        self.person_name = person_name
    
    def _load_one_facenet(self, path, resize_size=None, prewhiten_img=True):
        img = Image.open(path).convert("RGB")
    
            
        if resize_size:
            img = img.resize((resize_size, resize_size))
            
        img = (np.array(img)).astype(np.float32)
        
        if prewhiten_img:
            img = prewhiten(img)
            
        return img
        
    def _load_folder_for_facenet(self, folder, resize_size=None):
        paths_list = glob.glob(os.path.join(folder, "*"))
        final_imgs = []
        for img_path in paths_list:
            final_imgs.append(
                self._load_one_facenet(
                    img_path, prewhiten_img=False, resize_size=resize_size))
        
        final_imgs = np.array(final_imgs)
        mean, std = np.mean(final_imgs), np.std(final_imgs)
        final_imgs = prewhiten(np.array(final_imgs))
        return final_imgs, mean, std, paths_list
        
    def load_clean_from_folder(self, clean_folder, resize_size=160):
        self.clean_folder = clean_folder
        self.clean_images, self.orig_mean, self.orig_std, self.orig_paths = self._load_folder_for_facenet(
            clean_folder, resize_size=resize_size)
    
    def _undo_preprocess(self, images):
        restored_images = images.copy()
        restored_images  *= self.orig_std
        restored_images += self.orig_mean
        restored_images = np.clip(restored_images, 0.0, 255.0)
        return np.uint8(restored_images)
    
    def _compute_embeddings(self, model, images):
        return model.predict(np.array(images), batch_size=len(images))
    
    def get_clean_for_display(self):
        return self._undo_preprocess(self.clean_images)
    
    def compute_clean_embeddings_with_model(self, model):
        self.clean_embeddings = self._compute_embeddings(model, self.clean_images)

In [4]:
class MyModel:
    def __init__(self, model_path, inputs=None, sess=None):
        if inputs is None:
            self.model_inputs = tf.placeholder(tf.float32, shape=(None, 160, 160, 3))
        else:
            self.model_inputs = inputs
        
        vars_before = tf.global_variables()
        self.net, self.endpoints = inference(
            self.model_inputs, keep_probability=1.0, bottleneck_layer_size=512, phase_train=False)
        vars_after = tf.global_variables()
        
        model_name = dataset_to_model_name["vggface2"]
        saver = tf.train.Saver(list(set(vars_after) - set(vars_before)))
        
        if sess is None:
            self.sess = tf.Session()
        else:
            self.sess = sess
            
        saver.restore(self.sess, model_path)
    
    def predict(self, inputs, batch_size):
        return self.sess.run(self.net, feed_dict={self.model_inputs: inputs})
    
    def __del__(self):
        self.sess.close()

dataset_to_model_name = {
    "casia-webface": "20180408-102900",
    "vggface2": "20180402-114759"
}

dataset_to_ckpt_number = {
    "casia-webface": "90",
    "vggface2": "275"
}

def build_model(dataset_name, inputs=None, sess=None):
    model_name = dataset_to_model_name[dataset_name]
    model = MyModel(
        os.path.join(
            "/home/ivan/facenet/models",
            model_name,
            "model-{model_name}.ckpt-{ckpt_num}".format(
                model_name=dataset_to_model_name[dataset_name],
                ckpt_num=dataset_to_ckpt_number[dataset_name]
            )),
        inputs, 
        sess
    )
    return model

In [5]:
def path_for_id_clean(identity):
    if identity == "n000958":
        return "/data/vggface/test_perturbed_sampled/{id}/community_naive_mean/n000029/epsilon_0.0/png".format(id=identity)
    else:
        return "/data/vggface/test_perturbed_sampled/{id}/community_naive_mean/n000958/epsilon_0.0/png".format(id=identity)
    

In [6]:
def build_person(person_name, path_for_id_fn, model=None):
    person_a = PersonImages(person_name)
    person_a.load_clean_from_folder(path_for_id_fn(person_a.person_name))
    if model is not None:
        print("Computing embeddings for", person_name)
        person_a.compute_clean_embeddings_with_model(model)
    return person_a

In [7]:
vggface2_model = build_model("vggface2")
people = [
    build_person(person_name, path_for_id_clean, vggface2_model) \
    for person_name in os.listdir("/data/vggface/test_perturbed_sampled")
]
del vggface2_model

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Computing embeddings for n001781
Computing embeddings for n009232
Computing embeddings for n000958
Computing embeddings for n003356
Computing embeddings for n008655
Computing embeddings for n008613
Computing embeddings for n004658
Computing embeddings for n001683
Computing embeddings for n002647
Computing embeddings for n009288
Computing embeddings for n005427
Computing embeddings for n002763
Computing embeddings for n002503
Computing embeddings for n003215
Computing embeddings for n005359
Computing embeddings for n005303
Computing embeddings for n007548
Computing embeddings for n000029
Computing embeddings for n009114


In [8]:
def build_vggface_model(inputs, sess):
    return build_model("vggface2", inputs, sess)

def build_casiawebface_model(inputs, sess):
    return build_model("casia-webface", inputs, sess)

    
def attack_batch(
    build_model_fn, 
    input_images,
    target_vectors,
    learning_rate,
    epsilon,
    max_iters
):
    input_images = np.array(input_images)
    batch_size, orig_h, orig_w, orig_c = input_images.shape
    
    with tf.Graph().as_default():
        with tf.Session() as sess:
            targets_var = tf.get_variable(
                name="targets",
                shape=target_vectors.shape,
            )
            sess.run(tf.assign(targets_var, target_vectors))
            
            noisy_images_var = tf.get_variable(
                name="noisy_images",
                shape=input_images.shape
            )
            sess.run(tf.assign(noisy_images_var, input_images + tf.random.normal(input_images.shape, 0.0, 0.1)))

#             noise_var = tf.get_variable(
#                 name="adversarial_noise",
#                 shape=input_images.shape,
#                 initializer=tf.initializers.truncated_normal(
#                     mean=np.mean(input_images),
#                     std=np.std(input_images)
#                 )
#             )
            
                        
            randomized_images_plus_noise = tf.image.random_brightness(
                    noisy_images_var, 0.5)

            randomized_images_plus_noise = tf.image.random_crop(
                randomized_images_plus_noise, 
                [batch_size, orig_h - 10, orig_w - 10, 3]
            )
            
            randomized_images_plus_noise = tf.image.resize_images(
                randomized_images_plus_noise, [orig_h, orig_w])

            randomized_images_plus_noise += tf.random.normal(
                randomized_images_plus_noise.shape, 0.0, 0.75)
            
            model = build_model_fn(
                inputs=randomized_images_plus_noise, 
                sess=sess
            )
            
            model_outputs = tf.nn.l2_normalize(model.net, axis=0)
            targets = tf.nn.l2_normalize(targets_var, axis=0)
            print("model outputs shape", model_outputs.shape.as_list(), "targets shape", targets.shape.as_list())
            multiplied = tf.multiply(model_outputs, targets)
            print("multiplied shape", multiplied.shape.as_list())
            loss = tf.reduce_sum(multiplied)
            
            losses = []
            for i in range(max_iters):
                grads = tf.gradients(loss, [noisy_images_var])[0]
                assert grads is not None
                losses.append(sess.run(loss))
                sess.run([
                    tf.assign(
                        noisy_images_var,
                        noisy_images_var - learning_rate * tf.sign(grads)
                    ),
                    tf.assign(
                        noisy_images_var, 
                        tf.clip_by_value(
                            noisy_images_var, 
                            input_images - epsilon,
                            input_images + epsilon
                        ))
                ])
            return sess.run(noisy_images_var), losses

In [14]:
def attack_batch_cw(
    build_model_fn, 
    input_images,
    target_vectors,
    learning_rate,
    epsilon,
    max_iters
):
    input_images = np.array(input_images)
    batch_size, orig_h, orig_w, orig_c = input_images.shape
    print("input images mean {mean} and stddev {stddev}".format(mean=np.mean(input_images), stddev=np.std(input_images)))
    with tf.Graph().as_default():
        with tf.Session() as sess:
            targets_var = tf.get_variable(
                name="targets",
                shape=target_vectors.shape,
            )
            sess.run(tf.assign(targets_var, target_vectors))
            
            images_input_ph = tf.placeholder(
                tf.float32,
                name="input_images",
                shape=input_images.shape
            )
            
            noise_var = tf.get_variable(
                name="adversarial_noise",
                shape=input_images.shape,
                initializer=tf.initializers.truncated_normal(
                    mean=np.mean(input_images),
                    stddev=3 * np.std(input_images)
                )
            )
            sess.run(noise_var.initializer)
            
            images_plus_noise = images_input_ph + noise_var
            randomized_images_plus_noise = images_plus_noise
#             randomized_images_plus_noise = tf.image.random_brightness(
#                 images_plus_noise, 0.5)
#             print("shape of inputs", randomized_images_plus_noise.shape)
        
#             randomized_images_plus_noise = tf.image.random_crop(
#                 randomized_images_plus_noise, 
#                 [batch_size, orig_h - 10, orig_w - 10, 3]
#             )
#             print("shape of inputs", randomized_images_plus_noise.shape.as_list())

#             randomized_images_plus_noise = tf.image.resize_images(
#                 randomized_images_plus_noise, [orig_h, orig_w])
#             print("shape of inputs", randomized_images_plus_noise.shape.as_list())
            
#             randomized_images_plus_noise += tf.random.normal(
#                 randomized_images_plus_noise.shape, 0.0, 0.75)
#             print("shape of inputs", randomized_images_plus_noise.shape.as_list())
            
#             randomized_images_plus_noise = tf.clip_by_value(
#                 randomized_images_plus_noise, input_images - epsilon, input_images + epsilon)
#             print("shape of inputs", randomized_images_plus_noise.shape.as_list())
            
            model = build_model_fn(
                inputs=randomized_images_plus_noise, 
                sess=sess
            )
            
            model_outputs = tf.nn.l2_normalize(model.net, axis=1)
            targets = tf.nn.l2_normalize(targets_var, axis=1)
            loss = tf.losses.cosine_distance(targets, model_outputs, axis=1)
            print("loss shape", loss.shape)
        
            total_loss = loss #+ 1e-6 * tf.nn.l2_loss(noise_var)
            
            vars_before = set(tf.global_variables())
            train_op = tf.train.AdamOptimizer(learning_rate).minimize(total_loss, var_list=[noise_var])
            vars_after = set(tf.global_variables())
            sess.run([v.initializer for v in list(vars_after - vars_before)])
            
            
            losses = []
            for i in range(max_iters):
                print("mean of noise var", sess.run(tf.reduce_mean(noise_var)))
                print("mean of input", 
                      sess.run(
                          tf.reduce_mean(randomized_images_plus_noise), feed_dict={images_input_ph: input_images}))
                print("loss", 
                      sess.run(
                          loss, feed_dict={images_input_ph: input_images}))
                print("targets", sess.run(targets_var))
                print("input images mean {mean} and stddev {stddev}".format(mean=np.mean(input_images), stddev=np.std(input_images)))

                loss_value, total_loss_value, _ = sess.run(
                    [loss, total_loss, train_op], feed_dict={images_input_ph: input_images})
                print(i, loss_value, total_loss_value)
                assert not np.isnan(loss_value), "Loss_value is nan"
                losses.append(loss_value)
            
            final_imgs = sess.run(
                tf.clip_by_value(images_plus_noise, input_images - epsilon, input_images + epsilon),
                feed_dict={images_input_ph: input_images}
            )
            
            return final_imgs, losses

In [15]:
def generate_decoys(
    people_list,
    attack_strategy,
    model_to_attack,
    learning_rate,
    epsilon,
    max_iters
):
    if model_to_attack == "vggface2":
        model_build_fn = build_vggface_model
    elif model_to_attack == "casia-webface":
        model_build_fn = build_casiawebface_model
        
    for person in tqdm(people_list):
        num, height, width, channels = person.clean_images.shape
        person.adversarial_images = np.zeros((2*(len(people_list) - 1), height, width, channels))
        indx = 0
        for other_person in tqdm(people_list):
            if person.person_name == other_person.person_name:
                continue
            target_vector = np.mean(other_person.clean_embeddings, axis=0)
            images_to_make_adversarial = person.clean_images[indx : indx + 2] 
            targets_for_images = np.array([target_vector for _ in range(len(images_to_make_adversarial))])
            person.adversarial_images[indx : indx + 2], losses = attack_batch(
                model_build_fn, 
                images_to_make_adversarial,
                targets_for_images,
                learning_rate,
                epsilon,
                max_iters
            )
            print("Done generating decoys by", person.person_name, "for", other_person.person_name)
            
            fig, ax = fig1, ax1 = plt.subplots()
            ax.plot(range(len(losses)), losses)
            plt.show()
            
            save_dest = os.path.join(
                "/data/vggface/test_perturbed_sampled",
                person.person_name,
                "{attack_strategy}_{model}".format(attack_strategy=attack_strategy, model=model_to_attack),
                other_person.person_name
            )
            
            save_path = os.path.join(save_dest, "epsilon_{}".format(epsilon), "png")
            os.makedirs(save_path, exist_ok=True)
            existing_files = os.listdir(save_path)

            # Clean up folder if need be
            if len(existing_files) > 0:
                for f in existing_files:
                    os.remove(os.path.join(save_path, f))
            
            for i in range(indx, indx + 2):
                orig_name = person.orig_paths[i].split("/")[-1]
                
                full_save_path = os.path.join(save_path, orig_name)
                print("Saving to", save_path)
                Image.fromarray(
                    person._undo_preprocess(person.adversarial_images[i])
                ).save(full_save_path)

            indx += 2

In [16]:
# generate_decoys(
#     people,
#     "mean",
#     "vggface2",
#     0.1,
#     0.1,
#     100
# )

In [17]:
def generate_decoys_bigger_batches(
    people_list,
    attack_strategy,
    model_to_attack,
    learning_rate,
    epsilon,
    max_iters
):
    if model_to_attack == "vggface2":
        model_build_fn = build_vggface_model
    elif model_to_attack == "casia-webface":
        model_build_fn = build_casiawebface_model
        
    for person in tqdm(people_list):
        num, height, width, channels = person.clean_images.shape
        person.adversarial_images = np.zeros((2*(len(people_list) - 1), height, width, channels))
        
        indx = 0
        
        images_to_make_adversarial = []
        targets_for_images = []
        
        for other_person in people_list:
            if person.person_name == other_person.person_name:
                continue
                
            current_chosen_indices = range(indx, indx + 2)
            images_to_make_adversarial.extend(np.take(
                person.clean_images, current_chosen_indices, axis=0))
            
            target_vector = np.mean(other_person.clean_embeddings, axis=0)
            targets_for_images.extend(np.array([
                target_vector for _ in range(len(current_chosen_indices))]))
            
            indx += 2
            
        images_to_make_adversarial = np.array(images_to_make_adversarial)
        
        all_adversarial_images, losses = attack_batch_cw(
            model_build_fn, 
            np.array(images_to_make_adversarial),
            np.array(targets_for_images),
            learning_rate,
            epsilon,
            max_iters
        )
            
        fig, ax = plt.subplots()
        ax.plot(range(len(losses)), losses)
        plt.show()
        
        indx = 0
        for other_person in people_list:
            if person.person_name == other_person.person_name:
                continue
            
            save_dest = os.path.join(
                "/data/vggface/test_perturbed_sampled",
                person.person_name,
                "{attack_strategy}_{model}".format(attack_strategy=attack_strategy, model=model_to_attack),
                other_person.person_name
            )
            
            save_path = os.path.join(save_dest, "epsilon_{}".format(epsilon), "png")
            os.makedirs(save_path, exist_ok=True)
            existing_files = os.listdir(save_path)

            # Clean up folder if need be
            if len(existing_files) > 0:
                for f in existing_files:
                    os.remove(os.path.join(save_path, f))
            
            for i in range(indx, indx + 2):
                orig_name = person.orig_paths[i].split("/")[-1]
                person.adversarial_images[i] = all_adversarial_images[i]
                full_save_path = os.path.join(save_path, orig_name)
                print("Saving to", save_path)
                Image.fromarray(
                    person._undo_preprocess(person.adversarial_images[i])
                ).save(full_save_path)

            indx += 2

In [18]:
generate_decoys_bigger_batches(
    people,
    "mean",
    "vggface2",
    0.1,
    0.1,
    100
)

  0%|          | 0/19 [00:00<?, ?it/s]

input images mean 9.97825910786787e-09 and stddev 1.0000001192092896
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
loss shape ()
mean of noise var 0.0012735219
mean of input 0.0012735314
loss 1.0091147
targets [[ 0.25508723  0.3961168   1.215363   ... -0.37649658 -1.2318215
   0.46316475]
 [ 0.25508723  0.3961168   1.215363   ... -0.37649658 -1.2318215
   0.46316475]
 [ 2.0642266   0.40784636 -0.9280291  ... -0.40743008  0.05588722
  -1.1745586 ]
 ...
 [ 1.2804402   0.8868904  -1.2757783  ... -0.665468    0.2846522
  -0.74040914]
 [ 0.6271096   0.517079    0.23785053 ...  0.02542643 -0.18105283
  -0.497792  ]
 [ 0.6271096   0.517079    0.23785053 ...  0.02542643 -0.18105283
  -0.497792  ]]
input images mean 9.97825910786787e-09 and stddev 1.0000001192092896
0 1.0091147 1.0091147
mean of noise var 0.0016066105
mean of input 0.0016066219
loss 0.7661517
targets [[ 0.255087

mean of noise var 0.0038041016
mean of input 0.0038041077
loss 0.18682724
targets [[ 0.25508723  0.3961168   1.215363   ... -0.37649658 -1.2318215
   0.46316475]
 [ 0.25508723  0.3961168   1.215363   ... -0.37649658 -1.2318215
   0.46316475]
 [ 2.0642266   0.40784636 -0.9280291  ... -0.40743008  0.05588722
  -1.1745586 ]
 ...
 [ 1.2804402   0.8868904  -1.2757783  ... -0.665468    0.2846522
  -0.74040914]
 [ 0.6271096   0.517079    0.23785053 ...  0.02542643 -0.18105283
  -0.497792  ]
 [ 0.6271096   0.517079    0.23785053 ...  0.02542643 -0.18105283
  -0.497792  ]]
input images mean 9.97825910786787e-09 and stddev 1.0000001192092896
12 0.18682724 0.18682724
mean of noise var 0.0038546855
mean of input 0.003854694
loss 0.17092633
targets [[ 0.25508723  0.3961168   1.215363   ... -0.37649658 -1.2318215
   0.46316475]
 [ 0.25508723  0.3961168   1.215363   ... -0.37649658 -1.2318215
   0.46316475]
 [ 2.0642266   0.40784636 -0.9280291  ... -0.40743008  0.05588722
  -1.1745586 ]
 ...
 [ 1.280

  0%|          | 0/19 [00:18<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
with tf.Session() as sess:
    a = [[1., 2., 3.], [4., 5., 6.]]
    b = [[7., 8., 9.], [10., 11., 12.]]
    print("tensorflow computed", 
          sess.run(tf.losses.cosine_distance(
              tf.nn.l2_normalize(a, axis=1), 
              tf.nn.l2_normalize(b, axis=1), 
              axis=1)))
    print("shape of normalize a", tf.nn.l2_normalize(a, axis=0).shape.as_list())
    print("shape of normalize b", tf.nn.l2_normalize(b, axis=0).shape.as_list())
    print("shape of multiply", tf.multiply(
                  tf.nn.l2_normalize(a, axis=0), 
                  tf.nn.l2_normalize(b, axis=0)).shape.as_list())
    mult = tf.multiply(
                  tf.nn.l2_normalize(a, axis=1), 
                  tf.nn.l2_normalize(b, axis=1))
    print("mutiplied", sess.run(mult))
    print(
        "custom tensorflow",
        sess.run(1.0 - tf.reduce_sum(mult, axis=1)))


pd = pairwise_distances(a, b, metric="cosine")
print(pd)
print(pd[0, 0] + pd[1, 1])
print((pd[0, 0] + pd[1, 1])/2.0)
for a_i, b_i in zip(a, b):
    multiplied = (a_i/np.linalg.norm(a_i, ord=2)) * (b_i/np.linalg.norm(b_i, ord=2))
    print("multiplied", multiplied)
    print(1.0 - np.sum(multiplied))